## Installation
-  Run this only once

In [ ]:
!pip install gdown
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 37.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 59.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 KB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 KB 8.9 MB/s eta 0

## Data Loading

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np
from datetime import datetime

In [ ]:
SEED=42

In [ ]:
# !gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
# !unzip -o "data.zip"  -d  "/content"

# movie_10k_df = pd.read_csv("movies_min.csv")
# amazon_df =pd.read_csv('amazon_min.csv')
# book_df=pd.read_csv("book_min.csv")
# food_df = pd.read_csv('food_min.csv')
# ecommerce_df=pd.read_csv('ecom_min.csv')
# ciao_df=pd.read_csv('ciao_min.csv')

In [ ]:
TIMESTAMP=None

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct

Downloading...
From: https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
To: /content/data.zip
100% 4.16M/4.16M [00:00<00:00, 139MB/s]


In [ ]:
!unzip -o "data.zip"  -d  "/content"

Archive:  data.zip
  inflating: /content/amazon_min.csv  
  inflating: /content/book_min.csv   
  inflating: /content/ciao_min.csv   
  inflating: /content/ecom_min.csv   
  inflating: /content/food_min.csv   
  inflating: /content/movies_min.csv  


## Name of the file where user item interaction data avaiable?
- Column name should be in following name and order
- [ **itemID, userID , rating , timestamp** ]
- No issue if there is no column for **timestamp**

In [ ]:
filename="movies_min.csv" #enter your filename with file path

### Run below cell

In [ ]:
rating_df = pd.read_csv(filename)


In [ ]:
NUM_OF_COLS=rating_df.shape[1]


In [ ]:
import time 
if  'timestamp' in rating_df.columns:
  rating_df =rating_df[[ 'userID', 'itemID', 'rating','timestamp']]
  rating_df.columns =[ 'userID', 'itemID', 'rating','timestamp']
else:

  obj = time.gmtime(0)
  epoch = time.asctime(obj)
  curr_time = round(time.time()*1000)
  rating_df['timestamp']=curr_time
  rating_df =rating_df[[ 'userID', 'itemID', 'rating','timestamp']]
  rating_df.columns =[ 'userID', 'itemID', 'rating','timestamp']

In [ ]:
u_users=list(rating_df.userID.unique())

In [ ]:
u_items=list(rating_df.itemID.unique())

##### Code Setup

In [ ]:
import cornac
from recommenders.datasets.python_splitters import python_random_split,python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED



In [ ]:
df=rating_df.copy()
df=df[['userID','itemID','rating']]
train, test = python_stratified_split(df, 0.75)

train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)
bpr = cornac.models.BPR(
    k=200,
    max_iter=100,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=42
)



## Run BPR

In [ ]:
bpr.fit(train_set)

all_predictions = predict_ranking(bpr, train, usercol='userID', itemcol='itemID', remove_seen=True)

k = 10
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)

print(
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')
all_predictions=all_predictions[['userID','itemID']]

  0%|          | 0/100 [00:00<?, ?it/s]

Optimization finished!
Precision@K:	0.353234
Recall@K:	0.211474


#### Recommendations

In [ ]:
all_predictions.head(5)

,userID,itemID
72394,1,292
72395,1,312
72396,1,310
72397,1,284
72398,1,316
